1， reshape(-1)

In [2]:
import torch

In [8]:
batch = 5
channel_num = 3
width = 2
height = 2
tensor1 = torch.randn((batch, channel_num, width, height))
print(tensor1.shape)
#print(tensor1)
tensor2 = tensor1.reshape(-1) #由原来的四个维度变成了一个维度，相当于把四维tensor，拉升成为了一个一维tensor
print(tensor2.shape)
print(tensor2)

torch.Size([5, 3, 2, 2])
torch.Size([60])
tensor([ 0.2291, -0.3142,  0.5660,  0.1660, -0.6822, -0.8333, -0.4968, -0.4572,
        -0.2140, -1.2602,  0.0688,  0.7145, -0.7998,  1.2341,  0.1150,  2.4299,
         0.5717, -1.5586,  0.6175,  1.1083,  0.4488,  0.6110,  0.4972, -0.4272,
        -1.1512, -0.8648, -0.9167,  0.9993, -0.8286,  0.7657,  1.0027,  0.7891,
         0.3258, -0.0742,  0.5012, -1.4614,  0.9839,  1.7257,  0.3617, -1.3706,
         0.5814, -1.0665,  0.5048, -1.3078,  0.5859, -0.7169, -1.5131,  0.2691,
         0.4479,  2.2167, -0.8810, -0.9193,  0.7564, -0.0050, -4.0736, -1.2540,
         1.4856,  1.0118,  0.9243,  0.2119])


# numpy中的reshape

In [7]:
import numpy as np
z = np.array([[1, 2, 3, 4],[5, 6, 7, 8],[9, 10, 11, 12],[13, 14, 15, 16]])
print(z.shape)
z_flatten = z.reshape(-1) #将z变成一个维度的numpy数组
print(z_flatten.shape)
print(z_flatten)
z_col = z.reshape(-1, 1) #将z变成n行1列的2维数组
print(z_col.shape)

(4, 4)
(16,)
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
(16, 1)


# pytorch 中retain_graph==True的作用

https://blog.csdn.net/qq_39861441/article/details/104129368

总的来说进行一次backward之后，计算图会被清除，计算图中各节点的值依然是存在的，只是计算图被清除了。这样进行第二次backward会报错，如果加上retain_graph==True后,可以再来一次backward。
 
retain_graph参数的作用
官方定义：

retain_graph (bool, optional) – If False, the graph used to compute the grad will be freed. Note that in nearly all cases setting this option to True is not needed and often can be worked around in a much more efficient way. Defaults to the value of create_graph.

大意是如果设置为False，计算图中的中间变量在计算完后就会被释放。但是在平时的使用中这个参数默认都为False从而提高效率，和creat_graph的值一样。

具体看一个例子理解：

假设一个我们有一个输入x，y = x **2, z = y*4，然后我们有两个输出，一个output_1 = z.mean()，另一个output_2 = z.sum()。然后我们对两个output执行backward。

In [10]:
import torch
x = torch.randn((1,4),dtype=torch.float32,requires_grad=True)
y = x ** 2
z = y * 4
print(x)
print(y)
print(z)
loss1 = z.mean()
loss2 = z.sum()
print(loss1,loss2)
loss1.backward()    # 这个代码执行正常，但是执行完中间变量都free了，所以下一个出现了问题
print(loss1,loss2)
print(x)
print(y)
print(z)
loss2.backward()    # 这时会引发错误

tensor([[-0.3943,  1.6039, -0.6963, -0.5955]], requires_grad=True)
tensor([[0.1554, 2.5725, 0.4849, 0.3547]], grad_fn=<PowBackward0>)
tensor([[ 0.6218, 10.2901,  1.9395,  1.4187]], grad_fn=<MulBackward0>)
tensor(3.5675, grad_fn=<MeanBackward0>) tensor(14.2701, grad_fn=<SumBackward0>)
tensor(3.5675, grad_fn=<MeanBackward0>) tensor(14.2701, grad_fn=<SumBackward0>)
tensor([[-0.3943,  1.6039, -0.6963, -0.5955]], requires_grad=True)
tensor([[0.1554, 2.5725, 0.4849, 0.3547]], grad_fn=<PowBackward0>)
tensor([[ 0.6218, 10.2901,  1.9395,  1.4187]], grad_fn=<MulBackward0>)


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

程序正常执行到第12行，所有的变量正常保存。但是在第13行报错：

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

分析：计算节点数值保存了，但是计算图x-y-z结构被释放了，而计算loss2的backward仍然试图利用x-y-z的结构，因此会报错。

因此需要retain_graph参数为True去保留中间参数从而两个loss的backward()不会相互影响。正确的代码应当把第11行以及之后改成

In [11]:
import torch
x = torch.randn((1,4),dtype=torch.float32,requires_grad=True)
y = x ** 2
z = y * 4
print(x)
print(y)
print(z)
loss1 = z.mean()
loss2 = z.sum()
print(loss1,loss2)
loss1.backward(retain_graph=True)    # 这个参数表示保留计算图，
print(loss1,loss2)
print(x)
print(y)
print(z)
loss2.backward()    # 因此这里就不会发生错误了

tensor([[ 0.4325, -0.0458,  0.9422, -0.6456]], requires_grad=True)
tensor([[0.1871, 0.0021, 0.8878, 0.4168]], grad_fn=<PowBackward0>)
tensor([[0.7483, 0.0084, 3.5510, 1.6673]], grad_fn=<MulBackward0>)
tensor(1.4938, grad_fn=<MeanBackward0>) tensor(5.9750, grad_fn=<SumBackward0>)
tensor(1.4938, grad_fn=<MeanBackward0>) tensor(5.9750, grad_fn=<SumBackward0>)
tensor([[ 0.4325, -0.0458,  0.9422, -0.6456]], requires_grad=True)
tensor([[0.1871, 0.0021, 0.8878, 0.4168]], grad_fn=<PowBackward0>)
tensor([[0.7483, 0.0084, 3.5510, 1.6673]], grad_fn=<MulBackward0>)
